In [1]:
import pandas as pd

# Extract data from CSV file
album_ratings = pd.read_csv('album_ratings-collabs_sep.csv')

# Select pertinent columns
album_ratings = album_ratings[['Album', 'Artist', 'Year', 'Type', 'Length (in minutes)', 'Total tracks', 'Starred tracks', 'Disliked tracks', 'My rating', 'Rank', 'Year of first listen', 'Year first heard artist', 'Listened to full discography?']]

# Filter to include only studio albums and if full discography has been reviewed
album_ratings = album_ratings.loc[album_ratings['Type'] == 'Studio'].loc[album_ratings['Listened to full discography?'] == 'Y'].reset_index(drop=True)

# Sort by artist then year of album release
album_ratings = album_ratings.sort_values(by=['Artist', 'Year']).reset_index(drop=True)


# Create dataframe for rating statistics by artist
ratings = album_ratings[['Artist', 'My rating']]
ratings = ratings.groupby('Artist')
ratings = ratings.describe()
ratings = ratings.T
ratings = ratings.reset_index(drop=True)
ratings = ratings.T

# After transposing the dataframe twice, the columns need to be assigned the correct name
ratings.columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']

# Select pertinent columns
ratings = ratings[['count', 'min', '50%', 'max']]


# Create dataframe for album ranking statistics by artist
ranks = album_ratings[['Artist', 'Rank']]
ranks = ranks.groupby('Artist')
ranks = ranks.describe()
ranks = ranks.T
ranks = ranks.reset_index(drop=True)
ranks = ranks.T

# After transposing the dataframe twice, the columns need to be assigned the correct name
ranks.columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']

# Select pertinent columns
ranks = ranks[['count', 'min', '50%', 'max']]


# Create dataframe for count of starred and disliked tracks per artist
tracks = album_ratings[['Artist', 'Starred tracks', 'Disliked tracks']]
tracks = tracks.groupby('Artist').sum()

# Merge the ratings, ranks, and tracks dataframes
combined = pd.merge(ratings, ranks, how='outer', left_index=True, right_index=True)
combined = pd.merge(combined, tracks, how='outer', left_index=True, right_index=True)

# Select pertinent columns and rename them
combined = combined[['count_x', 'min_x', '50%_x', 'max_x', 'min_y', 'Starred tracks', 'Disliked tracks']]
combined.columns = ['count', 'min_rating', 'median_rating', 'max_rating', 'high_rank', 'starred', 'disliked']

# Create new column by applying formula to calculate artist score
combined['score'] = combined['min_rating'] + combined['median_rating'] + combined['max_rating'] -\
                    combined['high_rank'] + ((combined['starred'] / combined['starred'].median())*10) -\
                    (combined['disliked']*10)

# Sort by score
combined = combined.sort_values(by=['score'], ascending = False)

# Create new dataframe that shows artist position in index
positions = combined.reset_index().reset_index()
positions['index'] = positions['index'] + 1
positions = positions.rename(columns = {'index' : 'position'}).set_index(['position', 'Artist'], drop=True)

positions

,,count,min_rating,median_rating,max_rating,high_rank,starred,disliked,score
position,Artist,,,,,,,,
1,Franz Ferdinand,5.0,94.17,98.000,100.00,1.0,37.0,0.0,310.144359
2,The Beatles,13.0,72.31,88.750,100.00,1.0,107.0,1.0,304.931795
3,Father John Misty,4.0,93.85,96.525,98.75,18.0,30.0,0.0,286.509615
4,Spoon,9.0,86.88,93.500,99.00,14.0,38.0,0.0,284.867179
5,St. Vincent,5.0,83.18,91.360,100.00,1.0,20.0,0.0,283.796410
6,Green Day,11.0,74.62,89.440,100.00,1.0,59.0,1.0,283.316410
7,The Strokes,5.0,82.00,90.000,100.00,1.0,22.0,0.0,282.282051
8,John Prine,18.0,77.50,85.655,99.00,14.0,60.0,0.0,278.924231
9,Muse,8.0,71.54,89.165,100.00,1.0,43.0,1.0,271.756282


In [2]:
# Export positions dataframe to CSV
positions.to_csv('favorite-artists.csv')